# Import Modules

In [ ]:
# Regular libraries
import numpy as np
import pandas as pd
import itertools

# Package libraries
import import_notebook
import code_3D
import code_bar
import code_heatmaps
import code_kernel
import code_scatter
import code_PCA
import code_other
import code_plotly
import code_utils

# Define Classes

In [ ]:
class Counts:
    '''
    *Counts* represents the output of reading a fastq file.
    
    Parameters
    ----------
    df : pandas dataframe
        Dataframe containing the counts per codon.
    
    start_position : int, default None
        First position in the protein sequence that will be used for the first column of the
        array. If a protein has been mutated only from residue 100-150, then if start_position = 100,
        the algorithm will trim the first 99 amino acids in the input sequence. The last 
        residue will be calculated based on the length of the input array. We have set the default value to 2
        because normally the Methionine in position 1 is not mutated.
         
    aminoacids : list, default None
        List of aminoacids (in order). Stop codon needs to be '*'.
        If none, it will use the index of the dataframe   
    
    '''
    def __init__(self, df, start_position=None, aminoacids=None):
        self.dataframe = df
        
        if start_position:
            self.start_position = start_position
            self.positions = np.arange(start_position,len(df.columns)+1)
        else: # if none, use the columns of the dataframe
            self.positions = list(df.columns)
            self.start_position = self.positions[0]
                                       
        if aminoacids:
            self.aminoacids = aminoacids
        else: # if aminoacids is none, use the index of the dataframe
            if code_utils._is_DNA(df):
                self.aminoacids = code_utils._translate_codons(df)
            else:
                self.aminoacids = list(df.index)
        # bar plots
    mean_counts = code_bar.plot_meancounts
    library_representation = code_bar.plot_library_representation
      

In [ ]:
class Screen:
    '''
    *Screen* represents a mutagenesis experiment. If you are doing deep scan 
    mutagenesis, then every amino acid in the protein has been mutated to every possible 
    amino acid. For example, if there was a leucine at position 2, then this leucine would
    be mutated to the other 19 naturally occurring amino acids. However, you can also use the 
    package if you only have a handful of amino acid substitutions.


    Parameters
    -----------
    dataset : array
        2D matrix containing the enrichment scores of the point mutants. Columns will contain the
        amino acid substitutions, rows will contain the enrichment for each residue in the protein sequence.
    
    sequence : str
        Protein sequence (columns) in 1 letter code format.
    
    aminoacids : list, default list('ACDEFGHIKLMNPQRSTVWY*')
        Amino acid substitutions (rows). Submit in the same order that is used for the array.
    
    start_position : int, default 2
        First position in the protein sequence that will be used for the first column of the
        array. If a protein has been mutated only from residue 100-150, then if start_position = 100,
        the algorithm will trim the first 99 amino acids in the input sequence. The last 
        residue will be calculated based on the length of the input array. We have set the default value to 2
        because normally the Methionine in position 1 is not mutated.
    
    secondary : list, optional
        This parameter is used to group the data by secondary structure. The format is 
        the name of the secondary structure multiplied by the residue length of that motif.
        Example : [['β1']*(8),['L1']*(7),['α1']*(9),...,].
    
    roc_df: Pandas dataframe, optional
        A dataframe that contains a column of variants labeled 'Variant' with a column labeled 'Class'
        containing the true class of that mutation. This can be used to compare enrichment scores to some label (such as 
        pathogenicity as found in a Cancer database) using ROC AUC.
    
    fillna : int, default 0
        How to replace NaN values.
    
    
    Attributes
    ------------
    dataframe : pandas dataframe
        Contains the enrichment scores, position, sequence.
    
    Other attributes are same as input parameters: dataset, aminoacids, start_position, roc_df, secondary

    '''

    def __init__(self, dataset, sequence, aminoacids=list('ACDEFGHIKLMNPQRSTVWY*'),
                 start_position=2, fillna=0, secondary=None, roc_df=None):
        self.dataset = np.array(dataset)
        self.aminoacids = aminoacids
        self.start_position = start_position
        self.end_position = len(self.dataset[0])+start_position
        self.sequence_raw = ''.join(sequence) #why I am doing this?
        self.sequence = code_utils._transform_sequence(
            self.dataset, self.sequence_raw, self.start_position)
        self.dataframe_stopcodons, self.dataframe = code_utils._transform_dataset(
            self.dataset, self.sequence, self.aminoacids, self.start_position, fillna)
        self.dataframe_SNV = code_utils._select_SNV(self.dataframe)
        self.dataframe_nonSNV = code_utils._select_nonSNV(self.dataframe)

        # Optional parameters
        self.roc_df = roc_df
        self.secondary = secondary
        if self.secondary is not None:
            self.secondary, self.secondary_dup = code_utils._transform_secondary(
                self.dataset, self.secondary, self.start_position, self.aminoacids)
    
        # Assert messages
        assert len(sequence) => self.dataset[0], 'Input sequence is not long enough'
        
    # Methods (Associated functions)
    
    # code_kernel
    kernel = code_kernel.plot_kernel
    histogram = code_kernel.plot_hist

    # code_heatmaps
    heatmap = code_heatmaps.plot_heatmap
    heatmap_rows = code_heatmaps.plot_heatmap_rows
    heatmap_columns = code_heatmaps.plot_heatmap_columns
    
    # code_bar
    mean = code_bar.plot_mean
    differential = code_bar.plot_meandifferential
    position = code_bar.plot_position
    
    # code_scatter
    scatter = code_scatter.plot_scatter
    
    # code_PCA
    correlation = code_PCA.plot_correlation
    individual_correlation = code_PCA.plot_individual_correlation
    group_correlation = code_PCA.plot_group_correlation
    pca = code_PCA.plot_pca
    
    # code_Other
    rank = code_other.plot_rank
    miniheatmap = code_other.plot_miniheatmap
    neighboreffect = code_other.plot_neighboreffect
    secondary_mean = code_other.plot_secondary
    roc = code_other.plot_roc
    cumulative = code_other.plot_cumulative
    
    # code_3D
    scatter_3D = code_3D.plot_scatter_3D
    scatter_3D_pdbprop = code_3D.plot_scatter_3D_pdbprop
    pymol = code_3D.plot_pymol
    
    # code_plotly
    rank_plotly = code_plotly.plot_rank_plotly 
    scatter_plotly = code_plotly.plot_scatter_plotly